### Extend and Merge Approach for DAGs

In [4]:
##### Psuedocode #####


# function ModifiedSSMOSP(G, s, t, c, bound):
#     for v in V:
#         Π[v] = {}
#     Π[s][0] = {(0, 0, null, null)}  // (reward, negative_weight, pred, edge)
    
#     for i = 1 to n-1:
#         for v in V:
#             for e = (u, v) in incoming_edges(v):
#                 Extend-&-Merge(Π[v], Π[u], e, bound)
    
#     return max(Π[t][n-1], key=lambda p: p[0])  // Return path with max reward

# function Extend-&-Merge(R, Q, e, bound):
#     for p in Q:
#         if e.weight >= 0:
#             q = (p[0] + e.weight, p[1], p, e)
#         else:
#             q = (p[0], p[1] - e.weight, p, e)
        
#         if q[1] <= bound:
#             pos = calculate_position(q)
#             if pos not in R or R[pos][0] < q[0]:
#                 R[pos] = q

# function calculate_position(q):
#     // Implementation depends on specific r values and c_min values
#     // This is a simplified version
#     return (int(log(q[0] + 1)), int(log(q[1] + 1)))

# function reconstruct_path(p):
#     path = []
#     while p[2] is not null:
#         path.append(p[3])
#         p = p[2]
#     return reversed(path)



In [10]:
# In this code we do not take the advantage of topological sorting for DAGs.
# The time complexity of the algorithm improves by a factor of n while applying topological sort.

import math
from collections import defaultdict

class Edge:
    def __init__(self, u, v, weight):
        self.u = u
        self.v = v
        self.weight = weight

def modified_ssmosp(G, s, t, bound):
    n = len(G)
    Pi = defaultdict(lambda: defaultdict(dict))
    # initializing for the source
    Pi[s][0][(0, 0)] = (0, 0, None, None)  # (reward, negative_weight, pred, edge)

    # This outer loop takes care of the fact that there would be atmost n-1 vertices in our path.
    # This outer loop would not be required when we apply topological sort in our approach.
    for i in range(1, n):
        for v in G:
            for e in G[v]:
                extend_and_merge(Pi[e.v][i], Pi[e.u][i-1], e, bound)
    
    if t not in Pi or n-1 not in Pi[t]:
        return None  # No valid path found
    
    best_path = max(Pi[t][n-1].values(), key=lambda p: p[0])
    return reconstruct_path(best_path)

def extend_and_merge(R, Q, e, bound):
    for p in Q.values():
        if e.weight >= 0:
            q = (p[0] + e.weight, p[1], p, e)
        else:
            q = (p[0], p[1] - e.weight, p, e)
        
        if q[1] <= bound:
            pos = calculate_position(q)
            if pos not in R or R[pos][0] < q[0]:
                R[pos] = q

def calculate_position(q):
    # This is a simplified approach to calculate position based on the reward and penalty of each edge.
    # Need to think of a better approach if possible...
    return (int(math.log(q[0] + 1)), int(math.log(q[1] + 1)))

def reconstruct_path(p):
    path = []
    while p[2] is not None:
        path.append(p[3])
        p = p[2]
    return list(reversed(path))

# Example usage
if __name__ == "__main__":
    # Create a sample graph
    G = defaultdict(list)
    G[0].append(Edge(0, 1, 5))
    G[0].append(Edge(0, 2, -2))
    G[1].append(Edge(1, 3, 3))
    G[2].append(Edge(2, 1, 1))
    G[2].append(Edge(2, 3, 6))
    G[3].append(Edge(3, 4, -1))
    
    s = 0  # source
    t = 4  # target
    bound = 3  # maximum allowed sum of negative weights
    
    best_path = modified_ssmosp(G, s, t, bound)
    
    if best_path:
        print("Best path:")
        total_reward = 0
        total_negative = 0
        for edge in best_path:
            print(f"{edge.u} -> {edge.v} (weight: {edge.weight})")
            if edge.weight >= 0:
                total_reward += edge.weight
            else:
                total_negative -= edge.weight
        print(f"Total reward: {total_reward}")
        print(f"Total negative weight: {total_negative}")
    else:
        print("No valid path found")

Best path:
0 -> 1 (weight: 5)
1 -> 3 (weight: 3)
3 -> 4 (weight: -1)
Total reward: 8
Total negative weight: 1


### -------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
### Taking topological ordering into account


from collections import defaultdict, deque
import math

class Edge:
    def __init__(self, u, v, weight):
        self.u = u
        self.v = v
        self.weight = weight

class Graph:
    def __init__(self, num_vertices):
        self.V = num_vertices
        self.adj = defaultdict(list)   # adjacency list for outgoing edges
        self.incoming = defaultdict(list)  # adjacency list for incoming edges

    def add_edge(self, u, v, weight):
        edge = Edge(u, v, weight)
        self.adj[u].append(edge)
        self.incoming[v].append(edge)  # track incoming edges for each vertex

    def topological_sort(self):
        in_degree = {i: 0 for i in range(self.V)}
        for u in self.adj:
            for edge in self.adj[u]:
                in_degree[edge.v] += 1

        queue = deque([v for v in range(self.V) if in_degree[v] == 0])
        top_order = []

        while queue:
            u = queue.popleft()
            top_order.append(u)
            for edge in self.adj[u]:
                in_degree[edge.v] -= 1
                if in_degree[edge.v] == 0:
                    queue.append(edge.v)

        return top_order

def calculate_position(q):
    # Calculate position based on reward and negative weight (simplified version)
    return (int(math.log(q[0] + 1)), int(math.log(q[1] + 1)))

def ExtendAndMerge(R, Q, e, bound):
    for p in Q.values():
        if e.weight >= 0:
            q = (p[0] + e.weight, p[1], p, e)
        else:
            q = (p[0], p[1] - e.weight, p, e)
        
        if q[1] <= bound:
            pos = calculate_position(q)
            if pos not in R or R[pos][0] < q[0]:
                R[pos] = q

def reconstruct_path(p):
    path = []
    while p[2] is not None:
        path.append(p[3])
        p = p[2]
    return list(reversed(path))

def ModifiedSSMOSP(G, s, t, bound):
    # Step 1: Get topological order of the graph
    TopOrder = G.topological_sort()

    # Step 2: Initialize Π table
    Π = {v: {} for v in range(G.V)}
    Π[s][0] = (0, 0, None, None)  # (reward, negative_weight, pred, edge)
    
    # Step 3: Process vertices in topological order
    for v in TopOrder:
        for e in G.incoming[v]:
            ExtendAndMerge(Π[v], Π[e.u], e, bound)
    
    # Step 4: Find the path with the maximum reward to the target vertex
    if Π[t]:
        best_path = max(Π[t].values(), key=lambda p: p[0])
        return best_path[0], reconstruct_path(best_path)  # Return max reward and path
    else:
        return None, []  # No feasible path found within bound

# Example Usage
if __name__ == "__main__":
    # Create a graph
    G = Graph(6)
    G.add_edge(0, 1, 5)
    G.add_edge(0, 2, -2)
    G.add_edge(1, 3, 3)
    G.add_edge(2, 3, 4)
    G.add_edge(3, 4, -1)
    G.add_edge(4, 5, 6)
    
    # Define start, target, and bound
    start = 0
    target = 5
    bound = 2
    
    # Run ModifiedSSMOSP
    max_reward, path = ModifiedSSMOSP(G, start, target, bound)
    if path:
        print(f"Maximum Reward: {max_reward}")
        print("Path:", [(edge.u, edge.v, edge.weight) for edge in path])
    else:
        print("No feasible path found within the bound.")


Maximum Reward: 14
Path: [(0, 1, 5), (1, 3, 3), (3, 4, -1), (4, 5, 6)]


### Trying Different approaches for the calculate_position function

In [12]:
from collections import defaultdict, deque
import math

class Edge:
    def __init__(self, u, v, weight):
        self.u = u
        self.v = v
        self.weight = weight

class Graph:
    def __init__(self, num_vertices):
        self.V = num_vertices
        self.adj = defaultdict(list)   # adjacency list for outgoing edges
        self.incoming = defaultdict(list)  # adjacency list for incoming edges

    def add_edge(self, u, v, weight):
        edge = Edge(u, v, weight)
        self.adj[u].append(edge)
        self.incoming[v].append(edge)  # track incoming edges for each vertex

    def topological_sort(self):
        in_degree = {i: 0 for i in range(self.V)}
        for u in self.adj:
            for edge in self.adj[u]:
                in_degree[edge.v] += 1

        queue = deque([v for v in range(self.V) if in_degree[v] == 0])
        top_order = []

        while queue:
            u = queue.popleft()
            top_order.append(u)
            for edge in self.adj[u]:
                in_degree[edge.v] -= 1
                if in_degree[edge.v] == 0:
                    queue.append(edge.v)

        return top_order

# Alternative Bucketing Methods for `calculate_position`
def calculate_position_sqrt(q):
    # Square root bucketing approach
    return (int(math.sqrt(q[0] + 1)), int(math.sqrt(q[1] + 1)))

def calculate_position_double_log(q):
    # Double logarithmic bucketing approach
    reward_log = math.log(q[0] + 1)
    neg_weight_log = math.log(q[1] + 1)
    return (int(math.log(reward_log + 1)), int(math.log(neg_weight_log + 1)))

# Original or main `ExtendAndMerge` method, modifiable to use different bucketing
def ExtendAndMerge(R, Q, e, bound, calculate_position_func):
    for p in Q.values():
        if e.weight >= 0:
            q = (p[0] + e.weight, p[1], p, e)
        else:
            q = (p[0], p[1] - e.weight, p, e)
        
        if q[1] <= bound:
            pos = calculate_position_func(q)
            if pos not in R or R[pos][0] < q[0]:
                R[pos] = q

def reconstruct_path(p):
    path = []
    while p[2] is not None:
        path.append(p[3])
        p = p[2]
    return list(reversed(path))

def ModifiedSSMOSP(G, s, t, bound, calculate_position_func):
    # Step 1: Get topological order of the graph
    TopOrder = G.topological_sort()

    # Step 2: Initialize Π table
    Π = {v: {} for v in range(G.V)}
    Π[s][0] = (0, 0, None, None)  # (reward, negative_weight, pred, edge)
    
    # Step 3: Process vertices in topological order
    for v in TopOrder:
        for e in G.incoming[v]:
            ExtendAndMerge(Π[v], Π[e.u], e, bound, calculate_position_func)
    
    # Step 4: Find the path with the maximum reward to the target vertex
    if Π[t]:
        best_path = max(Π[t].values(), key=lambda p: p[0])
        return best_path[0], reconstruct_path(best_path)  # Return max reward and path
    else:
        return None, []  # No feasible path found within bound

# Example Usage
if __name__ == "__main__":
    # Create a graph
    G = Graph(6)
    G.add_edge(0, 1, 5)
    G.add_edge(0, 2, -2)
    G.add_edge(1, 3, 3)
    G.add_edge(2, 3, 4)
    G.add_edge(3, 4, -1)
    G.add_edge(4, 5, 6)
    
    # Define start, target, and bound
    start = 0
    target = 5
    bound = 2

    # Run ModifiedSSMOSP with different bucketing strategies
    print("Using Square Root Bucketing:")
    max_reward_sqrt, path_sqrt = ModifiedSSMOSP(G, start, target, bound, calculate_position_sqrt)
    if path_sqrt:
        print(f"Maximum Reward: {max_reward_sqrt}")
        print("Path:", [(edge.u, edge.v, edge.weight) for edge in path_sqrt])
    else:
        print("No feasible path found within the bound.")

    print("\nUsing Double Logarithmic Bucketing:")
    max_reward_double_log, path_double_log = ModifiedSSMOSP(G, start, target, bound, calculate_position_double_log)
    if path_double_log:
        print(f"Maximum Reward: {max_reward_double_log}")
        print("Path:", [(edge.u, edge.v, edge.weight) for edge in path_double_log])
    else:
        print("No feasible path found within the bound.")


Using Square Root Bucketing:
Maximum Reward: 14
Path: [(0, 1, 5), (1, 3, 3), (3, 4, -1), (4, 5, 6)]

Using Double Logarithmic Bucketing:
Maximum Reward: 14
Path: [(0, 1, 5), (1, 3, 3), (3, 4, -1), (4, 5, 6)]
